<a href="https://colab.research.google.com/github/Raoina/NLP-Learning-Journey/blob/main/Tasks/LSTM_MtoM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install needed libraries

In [ ]:
!pip install tensorflow numpy requests

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np
import os
import requests
import zipfile

# Download the dataset (English-French sentences)

In [3]:
data_url = 'https://storage.googleapis.com/download.tensorflow.org/data/fra-eng.zip'
data_path = 'fra-eng.zip'

In [4]:
if not os.path.exists('fra-eng.zip'):
    print("Downloading dataset...")
    r = requests.get(data_url)
    with open(data_path, 'wb') as f:
        f.write(r.content)

# Extract dataset

In [5]:
with zipfile.ZipFile(data_path, 'r') as zip_ref:
    zip_ref.extractall()


# File path of the extracted data

In [6]:
data_file = 'fra.txt'

In [7]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [8]:
with open(data_file, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

In [9]:
num_samples = 10000

In [11]:
for line in lines[:num_samples]:
    if '\t' not in line:
        continue
    input_text, target_text = line.split('\t')

    # Add start and end tokens to target sentence
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)

    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

# Sort characters for consistency

In [12]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

# Create token-index dictionaries
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])


Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59


# Initialize encoder input, decoder input and decoder target arrays

In [13]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

# Vectorize the data (One-hot encoding)
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # Decoder target data is ahead by one timestep
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0


# Define the model

In [14]:
latent_dim = 256

# Encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Full model for training

In [15]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    [encoder_input_data, decoder_input_data], decoder_target_data,
    batch_size=64,
    epochs=30,
    validation_split=0.2
)

# Inference models for prediction

# Encoder model for inference
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder model for inference
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to characters
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


Epoch 1/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 54s 401ms/step - accuracy: 0.0382 - loss: 1.1874 - val_accuracy: 0.0475 - val_loss: 1.1599
Epoch 2/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 80s 390ms/step - accuracy: 0.0471 - loss: 1.0302 - val_accuracy: 0.0464 - val_loss: 1.1720
Epoch 3/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 49s 392ms/step - accuracy: 0.0506 - loss: 1.0165 - val_accuracy: 0.0544 - val_loss: 1.1486
Epoch 4/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 83s 398ms/step - accuracy: 0.0545 - loss: 1.0026 - val_accuracy: 0.0653 - val_loss: 1.1599
Epoch 5/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 81s 391ms/step - accuracy: 0.0568 - loss: 0.9869 - val_accuracy: 0.0616 - val_loss: 1.1273
Epoch 6/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 49s 395ms/step - accuracy: 0.0624 - loss: 0.9722 - val_accuracy: 0.0703 - val_loss: 1.1085
Epoch 7/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 83s 405ms/step - accuracy: 0.0667 - loss: 0.9588 - val_accuracy: 0.0676 - val_loss: 1.1219
Epoch 8/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 81s 396ms/step - accuracy: 0.0714 - loss: 0

In [ ]:
def decode_sequence(input_seq):
    # Encode the input sequence to get the states
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence with only the start token
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length or find stop token.
        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence



# Test the model on some samples

In [20]:
for seq_index in range(10):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━